In [ ]:
from ipynb.fs.full.Database_Connections import *

In [ ]:
def get_wb_trip_opportunities(VSCVER_ID):
    import pandas as pd
    from datetime import datetime, timedelta, date, time

    #df = pd.read_csv("C:/Users/MBhutada.int/OneDrive - Chicago Transit Authority/Desktop/CTA_Mithilesh/CTA Assignments/Metra GTFS for Cubs/Excel and CSV files/Q-04 80 WB trips WK SA SU.csv")
    df = pd.read_csv(csv_name_wb_trips)
    
    df = df[['VSCVER_ID', 'TRIP_NO', 'TRIP_ID', 'START_PLACE', 'END_PLACE', 'START_TIME', 'END_TIME']]
    df['ST'] = df['START_TIME'].apply(lambda x: x/36000)

    def convert_time(element):
        minute = round((element - int(str(element)[:2])) * 60, 2)
        hour = float(str(element)[:2]) - 12

        if len(str(int(hour))) == 1:
            hour = '0' + str(int(hour))
        else:
            hour = str(int(hour))

        if minute % 1 == 00: # for minute part remainder = 0 (Ex. 9.0 or 17.0)
            if len(str(int(minute / 1))) == 1:
                minute = '0' + str(int(minute))
                element = hour + ':' + minute + ":00"  
            else:
                minute = str(int(minute))
                element = hour + ':' + minute + ":00"  
        else: # for minute part remainder != 0 (Ex. 9.5 or 17.5)
            if len(str(int(minute / 1))) == 1:
                minutes = '0' + str(int(minute))
                second = round((float(str(minute)[1:]) * 60), 2)
                element = hour + ':' + minutes + ":" + str(int(second))
            else:
                minutes = str(int(minute))
                second = round((float(str(minute)[2:]) * 60), 2) 
                element = hour + ':' + minutes + ":" + str(int(second))

        return element

    df['ST'] = df['ST'].apply(convert_time)
    df = df[df['START_PLACE'] == 'IrvBwy']

    df = df[(df['ST'] <= '23:59:59') & (df['VSCVER_ID'] == VSCVER_ID)]
    df = df.sort_values(['ST'])
    df = df.reset_index(drop = True)

    df['ST_timestamp'] = df['ST'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())

    trip_opportunities = []
    date = date(1, 1, 1)

    for i in range(len(df['ST_timestamp']) - 1):
        datetime1 = datetime.combine(date, df['ST_timestamp'][i])
        datetime2 = datetime.combine(date, df['ST_timestamp'][i+1])
        time_difference = datetime2 - datetime1
        mins = (datetime2 - datetime1).seconds/60

        if mins % 1 == 0:
            trip_opportunity = datetime1 + timedelta(minutes = mins/2)
            trip_opportunities.append(trip_opportunity)

        else:
            trip_opportunity = datetime1 + timedelta(minutes = mins/2 + 0.25)
            trip_opportunities.append(trip_opportunity)

    trip_opportunities = l = [x.time() for x in trip_opportunities]

    return trip_opportunities